##### tasks

<ol>
<li>for various time slices, for each planet in each run, parse for/calculate these values
<ol>
<li>eccentricity, inclination, jacobi distance, pericenter </li>
</ol>
<ol>
<li>for nearest and second nearest neighbor: ratios of semi-major axis, spacing in mutual hill radii, mass </li>
</ol>
</li>
<li>track for 3, 5, 10 timesteps
<ol>
<li>max, min, mean, final, initial, standard deviation for each value</li>
</ol>
</li>
</ol>

In [2]:
import pandas as pd
import numpy as np
import csv
from tqdm.notebook import tqdm
from numpy import *

In [ ]:
data = dict()

cols = ['Time','mass','a','e','i','omega','Omega','M','dist_jac','dist_cen','x','y','z','u','v','w']

pbar=tqdm(total=100)

for rn in range(1,101):
    pbar.update()
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = pd.read_csv(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el', skiprows=2, delimiter=r"\s+", names=cols) 
        
        for time in range(0,10):
            data[rn][planetnum][time] = dict()

            for col in cols:
                data[rn][planetnum][time][col] = dict()
                try:
                    data[rn][planetnum][time][col] = pl[col][time]
                except:
                    pass

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
var = ['e','i','pericenter','jacobi','mhr1','mhr2','a1_ratio','a2_ratio','mass1_ratio','mass2_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['a1_ratio'] = []
        te[rn2][planet]['a2_ratio'] = []
        te[rn2][planet]['mass1_ratio'] = []
        te[rn2][planet]['mass2_ratio'] = []

    
        for timestep in range (0,10):
            try:
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['dist_jac']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                ePL = float(data[rn2][planet][timestep]['e'])
                
                if (planet == 1):
                    m1 = float(data[rn2][planet+1][timestep]['mass'])
                    a1 = float(data[rn2][planet+1][timestep]['a'])

                    m2 = float(data[rn2][planet+2][timestep]['mass'])
                    a2 = float(data[rn2][planet+2][timestep]['a'])

                elif (planet == 10):
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet-2][timestep]['mass'])
                    a2 = float(data[rn2][planet-2][timestep]['a'])
                        
                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                
                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

### good version of code - two neighboring planets

In [15]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
var = ['e','i','pericenter','jacobi','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio']

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            try:
                te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                ePL = float(data[rn2][planet][timestep]['e'])
                
                if (planet == 1):
                    m1 = float(data[rn2][planet+1][timestep]['mass'])
                    a1 = float(data[rn2][planet+1][timestep]['a'])

                    m2 = float(data[rn2][planet+2][timestep]['mass'])
                    a2 = float(data[rn2][planet+2][timestep]['a'])

                elif (planet == 10):
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet-2][timestep]['mass'])
                    a2 = float(data[rn2][planet-2][timestep]['a'])
                        
                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                
                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv peri'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-15-9e43056acd45>:92: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-15-9e43056acd45>:93: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))


### largest mhr planets

In [24]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  

var = ['e','i','pericenter','jacobi','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio'] 
    
    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            try:
                te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                ePL = float(data[rn2][planet][timestep]['e'])
                
                
                fcmhr = []
                ps = []
                
                for p in range(1,11):
                    if p == planet:
                        pass
                    else:
                        ps.append(p)
                        fcmhr.append((np.abs(aPL-data[rn2][p][timestep]['a']))/((((massPL + data[rn2][p][timestep]['mass'])/3) ** (1/3)) * ((aPL + data[rn2][p][timestep]['a'])/2)))                        
                
                p1 = ps[fcmhr.index(np.max(fcmhr))]
                
                fcmhr.remove(np.max(fcmhr))
                ps.remove(p1)
                
                p2 = ps[fcmhr.index(np.max(fcmhr))]
                
                
                m1 = float(data[rn2][p1][timestep]['mass'])
                a1 = float(data[rn2][p1][timestep]['a'])

                m2 = float(data[rn2][p2][timestep]['mass'])
                a2 = float(data[rn2][p2][timestep]['a'])

                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                
                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv peri'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass
        
        
                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-24-afa5ddcc7f78>:97: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-24-afa5ddcc7f78>:98: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))


### most massive planets

In [20]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
# var = ['e','i','pericenter','jacobi','mhr1','a1_ratio','mass1_ratio'] 
var = ['e','i','pericenter','jacobi','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:    
                if (planet == lm):
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])

                elif (planet == lm2):
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])

                else:
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm2][0]['mass'])
                    a2 = float(data[rn2][lm2][timestep]['a'])
                    
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))

                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv peri'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-20-4978c815e41d>:112: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-20-4978c815e41d>:113: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))


In [21]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
# var = ['e','i','pericenter','jacobi','mhr1','a1_ratio','mass1_ratio'] 
var = ['e','i','pericenter','jacobi','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:    
                m1 = float(data[rn2][lm][0]['mass'])
                a1 = float(data[rn2][lm][timestep]['a'])

                m2 = float(data[rn2][lm2][0]['mass'])
                a2 = float(data[rn2][lm2][timestep]['a'])
                    
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))

                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv peri'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-21-a1dcc726676b>:95: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-21-a1dcc726676b>:96: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))


### 2nd and 3rd most massive

In [36]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
# var = ['e','i','pericenter','jacobi','mhr1','a1_ratio','mass1_ratio'] 
var = ['log e','i','pericenter','jacobi','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['log e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['log e'].append(float(np.log10(data[rn2][planet][timestep]['e'])))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:    
                if (planet == lm2):
                    m1 = float(data[rn2][lm3][0]['mass'])
                    a1 = float(data[rn2][lm3][timestep]['a'])

                    m2 = float(data[rn2][lm4][0]['mass'])
                    a2 = float(data[rn2][lm4][timestep]['a'])

                elif (planet == lm3):
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm4][0]['mass'])
                    a2 = float(data[rn2][lm4][timestep]['a'])

                else:
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])
                    
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))

                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['log e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['log e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv peri'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-36-18d03740f391>:117: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-36-18d03740f391>:118: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))


### 2nd a 3rd revised

In [17]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
# var = ['e','i','pericenter','jacobi','mhr1','a1_ratio','mass1_ratio'] 
var = ['e','i','pericenter','jacobi','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:    
                m1 = float(data[rn2][lm2][0]['mass'])
                a1 = float(data[rn2][lm2][timestep]['a'])

                m2 = float(data[rn2][lm3][0]['mass'])
                a2 = float(data[rn2][lm3][timestep]['a'])
                    
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))

                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv peri'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-17-45d230eeebb0>:102: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-17-45d230eeebb0>:103: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))


In [37]:
nested_keys = []
nested_keys.append('cv e')
nested_keys.append('cv a')
nested_keys.append('cv peri')
nested_keys.append('cv i')

var = ['log e','i','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio'] 

for v in var:
    nested_keys.append('initial ' + str(v))
    nested_keys.append('final ' + str(v))
    nested_keys.append('mean ' + str(v))
    nested_keys.append('min ' + str(v))
    nested_keys.append('max ' + str(v))
    nested_keys.append('sd ' + str(v))

    
with open('2-3 mm log e.csv', 'w') as f:
    header = 'run, planet, ' + ', '.join(map(str,nested_keys)) + ', end time, fate' + '\n'
    f.write(header)
    
    for r in list(sys.keys()):
        for p in list(sys[r].keys()):
            line = ''
            line += str(r) + ', ' + str(p)
            for va in nested_keys:
                line += ', ' + str(sys[r][p][va])
            line += ', ' + str(sys[r][p]['end time']) + ', ' + str(sys[r][p]['fate'])
            line += '\n'
            f.write(line)

#### 3 most massive planets

In [32]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
# var = ['e','i','pericenter','jacobi','mhr1','a1_ratio','mass1_ratio'] 
var = ['e','i','pericenter','jacobi','mhr1','mhr2','mhr3','log a1_ratio','log a2_ratio','log a3_ratio','log mass1_ratio','log mass2_ratio','log mass3_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['mhr3'] = []
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log a3_ratio'] = []
        te[rn2][planet]['log mass1_ratio'] = []
        te[rn2][planet]['log mass2_ratio'] = []
        te[rn2][planet]['log mass3_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:
                if planet == lm:
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])

                    m3 = float(data[rn2][lm4][0]['mass'])
                    a3 = float(data[rn2][lm4][timestep]['a'])
                    
                elif planet == lm2:
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])

                    m3 = float(data[rn2][lm4][0]['mass'])
                    a3 = float(data[rn2][lm4][timestep]['a'])
                    
                elif planet == lm3:
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm2][0]['mass'])
                    a2 = float(data[rn2][lm2][timestep]['a'])

                    m3 = float(data[rn2][lm4][0]['mass'])
                    a3 = float(data[rn2][lm4][timestep]['a'])
                    
                else:
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm2][0]['mass'])
                    a2 = float(data[rn2][lm2][timestep]['a'])

                    m3 = float(data[rn2][lm3][0]['mass'])
                    a3 = float(data[rn2][lm3][timestep]['a'])
                    
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                mhr3 = (np.abs(aPL-a3))/((((massPL + m3)/3) ** (1/3)) * ((aPL + a3)/2))

                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['mhr3'].append(float(mhr3))
                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log a3_ratio'].append(float(np.log10(aPL/a3)))
                te[rn2][planet]['log mass1_ratio'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mass2_ratio'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['log mass3_ratio'].append(float(np.log10(massPL/m3)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv peri'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-32-ba2f91fa77e6>:139: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-32-ba2f91fa77e6>:140: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
<ipython-input-32-ba2f91fa77e6>:141: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a3_ratio'].append(float(np.log10(aPL/a3)))


### np + 1-3mm

In [9]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
var = ['e','i','pericenter','jacobi','mhr mm1','mhr mm2','mhr mm3','log aratio mm1','log aratio mm2','log aratio mm3',
       'log mratio mm1','log mratio mm2','log mratio mm3', 'mhr npi', 'mhr npo', 'log aratio npi', 'log aratio npo',
       'log mratio npi', 'log mratio npo'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr mm1'] = []
        te[rn2][planet]['mhr mm2'] = []
        te[rn2][planet]['mhr mm3'] = []
        te[rn2][planet]['log aratio mm1'] = []
        te[rn2][planet]['log aratio mm2'] = []
        te[rn2][planet]['log aratio mm3'] = []
        te[rn2][planet]['log mratio mm1'] = []
        te[rn2][planet]['log mratio mm2'] = []
        te[rn2][planet]['log mratio mm3'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:
                if planet == lm:
                    m1 = float(data[rn2][lm2][0]['mass'])
                    a1 = float(data[rn2][lm2][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])

                    m3 = float(data[rn2][lm4][0]['mass'])
                    a3 = float(data[rn2][lm4][timestep]['a'])
                    
                elif planet == lm2:
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm3][0]['mass'])
                    a2 = float(data[rn2][lm3][timestep]['a'])

                    m3 = float(data[rn2][lm4][0]['mass'])
                    a3 = float(data[rn2][lm4][timestep]['a'])
                    
                elif planet == lm3:
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm2][0]['mass'])
                    a2 = float(data[rn2][lm2][timestep]['a'])

                    m3 = float(data[rn2][lm4][0]['mass'])
                    a3 = float(data[rn2][lm4][timestep]['a'])
                    
                else:
                    m1 = float(data[rn2][lm][0]['mass'])
                    a1 = float(data[rn2][lm][timestep]['a'])

                    m2 = float(data[rn2][lm2][0]['mass'])
                    a2 = float(data[rn2][lm2][timestep]['a'])

                    m3 = float(data[rn2][lm3][0]['mass'])
                    a3 = float(data[rn2][lm3][timestep]['a'])
                    
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                mhr3 = (np.abs(aPL-a3))/((((massPL + m3)/3) ** (1/3)) * ((aPL + a3)/2))

                te[rn2][planet]['mhr mm1'].append(float(mhr1))
                te[rn2][planet]['mhr mm2'].append(float(mhr2))
                te[rn2][planet]['mhr mm3'].append(float(mhr3))
                te[rn2][planet]['log aratio mm1'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log aratio mm2'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log aratio mm3'].append(float(np.log10(aPL/a3)))
                te[rn2][planet]['log mratio mm1'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mratio mm2'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['log mratio mm3'].append(float(np.log10(massPL/m3)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

            
            
te2 = dict()
for rn2 in range(1,101):
    te2[rn2] = dict()
    
    for planet in range (1,11):
        te2[rn2][planet] = dict()
        te2[rn2][planet]['mhr npi'] = []
        te2[rn2][planet]['mhr npo'] = []
        te2[rn2][planet]['log aratio npi'] = []
        te2[rn2][planet]['log aratio npo'] = []
        te2[rn2][planet]['log mratio npi'] = []
        te2[rn2][planet]['log mratio npo'] = []

    
        for timestep in range(len(data[rn2][planet])):
            try:
                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                ePL = float(data[rn2][planet][timestep]['e'])
                
                if (planet == 1):
                    m1 = float(data[rn2][planet+1][timestep]['mass'])
                    a1 = float(data[rn2][planet+1][timestep]['a'])

                    m2 = float(data[rn2][planet+2][timestep]['mass'])
                    a2 = float(data[rn2][planet+2][timestep]['a'])

                elif (planet == 10):
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet-2][timestep]['mass'])
                    a2 = float(data[rn2][planet-2][timestep]['a'])
                        
                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                mhri = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhro = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                
                te2[rn2][planet]['mhr npi'].append(float(mhri))
                te2[rn2][planet]['mhr npo'].append(float(mhro))
                te2[rn2][planet]['log aratio npi'].append(float(np.log10(aPL/a1)))
                te2[rn2][planet]['log aratio npo'].append(float(np.log10(aPL/a2)))
                te2[rn2][planet]['log mratio npi'].append(float(np.log10(massPL/m1)))
                te2[rn2][planet]['log mratio npo'].append(float(np.log10(massPL/m2)))                
                
            except:
                pass

            
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv pericenter'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                sys[rn2][planet]['initial ' + str(v)] = te2[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te2[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te2[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te2[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te2[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te2[rn2][planet][str(v)])

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-9-807a49fbdb85>:140: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log aratio mm1'].append(float(np.log10(aPL/a1)))
<ipython-input-9-807a49fbdb85>:141: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log aratio mm2'].append(float(np.log10(aPL/a2)))
<ipython-input-9-807a49fbdb85>:142: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log aratio mm3'].append(float(np.log10(aPL/a3)))
<ipython-input-9-807a49fbdb85>:200: RuntimeWarning: invalid value encountered in log10
  te2[rn2][planet]['log aratio npi'].append(float(np.log10(aPL/a1)))
<ipython-input-9-807a49fbdb85>:201: RuntimeWarning: invalid value encountered in log10
  te2[rn2][planet]['log aratio npo'].append(float(np.log10(aPL/a2)))


In [10]:
nested_keys = []
nested_keys.append('cv e')
nested_keys.append('cv a')
nested_keys.append('cv pericenter')
nested_keys.append('cv i')


var = ['e','i','pericenter','jacobi','mhr mm1','mhr mm2','mhr mm3','log aratio mm1','log aratio mm2','log aratio mm3',
       'log mratio mm1','log mratio mm2','log mratio mm3', 'mhr npi', 'mhr npo', 'log aratio npi', 'log aratio npo',
       'log mratio npi', 'log mratio npo']  


for v in var:
    nested_keys.append('initial ' + str(v))
    nested_keys.append('final ' + str(v))
    nested_keys.append('mean ' + str(v))
    nested_keys.append('min ' + str(v))
    nested_keys.append('max ' + str(v))
    nested_keys.append('sd ' + str(v))

    
with open('1-3mm & np.csv', 'w') as f:
    header = 'run, planet, ' + ', '.join(map(str,nested_keys)) + ', end time, fate' + '\n'
    f.write(header)
    
    for r in list(sys.keys()):
        for p in list(sys[r].keys()):
            line = ''
            line += str(r) + ', ' + str(p)
            for va in nested_keys:
                line += ', ' + str(sys[r][p][va])
            line += ', ' + str(sys[r][p]['end time']) + ', ' + str(sys[r][p]['fate'])
            line += '\n'
            f.write(line)

#### not mass adjusted
- mm1 is mm1 for all planets in the system

In [11]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
var = ['e','i','pericenter','jacobi','mhr mm1','mhr mm2','mhr mm3','log aratio mm1','log aratio mm2','log aratio mm3',
       'log mratio mm1','log mratio mm2','log mratio mm3', 'mhr npi', 'mhr npo', 'log aratio npi', 'log aratio npo',
       'log mratio npi', 'log mratio npo'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
                
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr mm1'] = []
        te[rn2][planet]['mhr mm2'] = []
        te[rn2][planet]['mhr mm3'] = []
        te[rn2][planet]['log aratio mm1'] = []
        te[rn2][planet]['log aratio mm2'] = []
        te[rn2][planet]['log aratio mm3'] = []
        te[rn2][planet]['log mratio mm1'] = []
        te[rn2][planet]['log mratio mm2'] = []
        te[rn2][planet]['log mratio mm3'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            ePL = float(data[rn2][planet][timestep]['e'])
                
            try:
                m1 = float(data[rn2][lm][0]['mass'])
                a1 = float(data[rn2][lm][timestep]['a'])

                m2 = float(data[rn2][lm2][0]['mass'])
                a2 = float(data[rn2][lm2][timestep]['a'])

                m3 = float(data[rn2][lm3][0]['mass'])
                a3 = float(data[rn2][lm3][timestep]['a'])
                    
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                mhr3 = (np.abs(aPL-a3))/((((massPL + m3)/3) ** (1/3)) * ((aPL + a3)/2))

                te[rn2][planet]['mhr mm1'].append(float(mhr1))
                te[rn2][planet]['mhr mm2'].append(float(mhr2))
                te[rn2][planet]['mhr mm3'].append(float(mhr3))
                te[rn2][planet]['log aratio mm1'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log aratio mm2'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log aratio mm3'].append(float(np.log10(aPL/a3)))
                te[rn2][planet]['log mratio mm1'].append(float(np.log10(massPL/m1)))
                te[rn2][planet]['log mratio mm2'].append(float(np.log10(massPL/m2)))
                te[rn2][planet]['log mratio mm3'].append(float(np.log10(massPL/m3)))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

            
            
te2 = dict()
for rn2 in range(1,101):
    te2[rn2] = dict()
    
    for planet in range (1,11):
        te2[rn2][planet] = dict()
        te2[rn2][planet]['mhr npi'] = []
        te2[rn2][planet]['mhr npo'] = []
        te2[rn2][planet]['log aratio npi'] = []
        te2[rn2][planet]['log aratio npo'] = []
        te2[rn2][planet]['log mratio npi'] = []
        te2[rn2][planet]['log mratio npo'] = []

    
        for timestep in range(len(data[rn2][planet])):
            try:
                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                ePL = float(data[rn2][planet][timestep]['e'])
                
                if (planet == 1):
                    m1 = float(data[rn2][planet+1][timestep]['mass'])
                    a1 = float(data[rn2][planet+1][timestep]['a'])

                    m2 = float(data[rn2][planet+2][timestep]['mass'])
                    a2 = float(data[rn2][planet+2][timestep]['a'])

                elif (planet == 10):
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet-2][timestep]['mass'])
                    a2 = float(data[rn2][planet-2][timestep]['a'])
                        
                else:
                    m1 = float(data[rn2][planet-1][timestep]['mass'])
                    a1 = float(data[rn2][planet-1][timestep]['a'])

                    m2 = float(data[rn2][planet+1][timestep]['mass'])
                    a2 = float(data[rn2][planet+1][timestep]['a'])

                mhri = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhro = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                
                te2[rn2][planet]['mhr npi'].append(float(mhri))
                te2[rn2][planet]['mhr npo'].append(float(mhro))
                te2[rn2][planet]['log aratio npi'].append(float(np.log10(aPL/a1)))
                te2[rn2][planet]['log aratio npo'].append(float(np.log10(aPL/a2)))
                te2[rn2][planet]['log mratio npi'].append(float(np.log10(massPL/m1)))
                te2[rn2][planet]['log mratio npo'].append(float(np.log10(massPL/m2)))                
                
            except:
                pass

            
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv pericenter'] = np.std(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['pericenter'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                sys[rn2][planet]['initial ' + str(v)] = te2[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te2[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te2[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te2[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te2[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te2[rn2][planet][str(v)])

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-11-6055d2e87b94>:109: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log aratio mm1'].append(float(np.log10(aPL/a1)))
<ipython-input-11-6055d2e87b94>:110: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log aratio mm2'].append(float(np.log10(aPL/a2)))
<ipython-input-11-6055d2e87b94>:111: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log aratio mm3'].append(float(np.log10(aPL/a3)))
<ipython-input-11-6055d2e87b94>:169: RuntimeWarning: invalid value encountered in log10
  te2[rn2][planet]['log aratio npi'].append(float(np.log10(aPL/a1)))
<ipython-input-11-6055d2e87b94>:170: RuntimeWarning: invalid value encountered in log10
  te2[rn2][planet]['log aratio npo'].append(float(np.log10(aPL/a2)))


In [12]:
nested_keys = []
nested_keys.append('cv e')
nested_keys.append('cv a')
nested_keys.append('cv pericenter')
nested_keys.append('cv i')


var = ['e','i','pericenter','jacobi','mhr mm1','mhr mm2','mhr mm3','log aratio mm1','log aratio mm2','log aratio mm3',
       'log mratio mm1','log mratio mm2','log mratio mm3', 'mhr npi', 'mhr npo', 'log aratio npi', 'log aratio npo',
       'log mratio npi', 'log mratio npo']  


for v in var:
    nested_keys.append('initial ' + str(v))
    nested_keys.append('final ' + str(v))
    nested_keys.append('mean ' + str(v))
    nested_keys.append('min ' + str(v))
    nested_keys.append('max ' + str(v))
    nested_keys.append('sd ' + str(v))

    
with open('1-3mm & np notma.csv', 'w') as f:
    header = 'run, planet, ' + ', '.join(map(str,nested_keys)) + ', end time, fate' + '\n'
    f.write(header)
    
    for r in list(sys.keys()):
        for p in list(sys[r].keys()):
            line = ''
            line += str(r) + ', ' + str(p)
            for va in nested_keys:
                line += ', ' + str(sys[r][p][va])
            line += ', ' + str(sys[r][p]['end time']) + ', ' + str(sys[r][p]['fate'])
            line += '\n'
            f.write(line)

### only mass ratios (all)

In [18]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=1)
        
        data[rn][planetnum]['mass'] = pl[1]
        
    
sys = dict()

for rn2 in range(1,101):
    sys[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p]['mass'])                
                
    lm = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm4)
    
    lm5 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm5)
    
    lm6 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm6)
    
    lm7 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm7)
    
    lm8 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm8)
    
    lm9 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm9)
    
    lm10 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm10)
    
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        
        massPL = float(data[rn2][planet]['mass'])
        
        sys[rn2][planet]['log mass1_ratio'] = float(np.log10(massPL/lm))
        sys[rn2][planet]['log mass2_ratio'] = float(np.log10(massPL/lm2))
        sys[rn2][planet]['log mass3_ratio'] = float(np.log10(massPL/lm3))
        sys[rn2][planet]['log mass4_ratio'] = float(np.log10(massPL/lm4))
        sys[rn2][planet]['log mass5_ratio'] = float(np.log10(massPL/lm5))
        sys[rn2][planet]['log mass6_ratio'] = float(np.log10(massPL/lm6))
        sys[rn2][planet]['log mass7_ratio'] = float(np.log10(massPL/lm7))
        sys[rn2][planet]['log mass8_ratio'] = float(np.log10(massPL/lm8))
        sys[rn2][planet]['log mass9_ratio'] = float(np.log10(massPL/lm9))
        sys[rn2][planet]['log mass10_ratio'] = float(np.log10(massPL/lm10))
        
        
        

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

In [19]:
nested_keys = ['log mass1_ratio','log mass2_ratio','log mass3_ratio','log mass4_ratio','log mass5_ratio','log mass6_ratio','log mass7_ratio','log mass8_ratio','log mass9_ratio','log mass10_ratio','fate'] 

with open('only mass ratios.csv', 'w') as f:
    header = 'run, planet, ' + ', '.join(map(str,nested_keys)) + '\n'
    f.write(header)
    
    for r in list(sys.keys()):
        for p in list(sys[r].keys()):
            
            line = ''
            line += str(r) + ', ' + str(p)
            for va in nested_keys:
                line += ', ' + str(sys[r][p][va])
            line += '\n'
            f.write(line)

### everything

#### minus redundant/possibly biased feats

In [24]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
            

    
    
sys = dict()
te = dict()


for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    am = []
    ps = []
                
    for p in range(1,11):
        ps.append(p)
        am.append(data[rn2][p][0]['mass'])                
                
    lm = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm)
        
    lm2 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm2)
    
    lm3 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm3)
    
    lm4 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm4)
    
    lm5 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm5)
    
    lm6 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm6)
    
    lm7 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm7)
    
    lm8 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm8)
    
    lm9 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm9)
    
    lm10 = ps[am.index(np.max(am))]
    am.remove(np.max(am))
    ps.remove(lm10)

        
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        massPL = float(data[rn2][planet][0]['mass'])
        
        sys[rn2][planet]['log mass1_ratio'] = float(np.log10(massPL/lm))
        sys[rn2][planet]['log mass2_ratio'] = float(np.log10(massPL/lm2))
        sys[rn2][planet]['log mass3_ratio'] = float(np.log10(massPL/lm3))
        sys[rn2][planet]['log mass4_ratio'] = float(np.log10(massPL/lm4))
        sys[rn2][planet]['log mass5_ratio'] = float(np.log10(massPL/lm5))
        sys[rn2][planet]['log mass6_ratio'] = float(np.log10(massPL/lm6))
        sys[rn2][planet]['log mass7_ratio'] = float(np.log10(massPL/lm7))
        sys[rn2][planet]['log mass8_ratio'] = float(np.log10(massPL/lm8))
        sys[rn2][planet]['log mass9_ratio'] = float(np.log10(massPL/lm9))
        sys[rn2][planet]['log mass10_ratio'] = float(np.log10(massPL/lm10))

        te[rn2][planet]['a'] = []
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['mhr3'] = []
        te[rn2][planet]['mhr4'] = []
        te[rn2][planet]['mhr5'] = []
        te[rn2][planet]['mhr6'] = []
        te[rn2][planet]['mhr7'] = []
        te[rn2][planet]['mhr8'] = []
        te[rn2][planet]['mhr9'] = []
        te[rn2][planet]['mhr10'] = []
        
        te[rn2][planet]['log a1_ratio'] = []
        te[rn2][planet]['log a2_ratio'] = []
        te[rn2][planet]['log a3_ratio'] = []
        te[rn2][planet]['log a4_ratio'] = []
        te[rn2][planet]['log a5_ratio'] = []
        te[rn2][planet]['log a6_ratio'] = []
        te[rn2][planet]['log a7_ratio'] = []
        te[rn2][planet]['log a8_ratio'] = []
        te[rn2][planet]['log a9_ratio'] = []
        te[rn2][planet]['log a10_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            te[rn2][planet]['a'].append(float(data[rn2][planet][timestep]['a']))
            te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
            te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
            
            massPL = float(data[rn2][planet][timestep]['mass'])
            aPL = float(data[rn2][planet][timestep]['a'])
            
            try:
                a1 = float(data[rn2][lm][timestep]['a'])
                a2 = float(data[rn2][lm2][timestep]['a'])
                a3 = float(data[rn2][lm3][timestep]['a'])
                a4 = float(data[rn2][lm4][timestep]['a'])
                a5 = float(data[rn2][lm5][timestep]['a'])
                a6 = float(data[rn2][lm6][timestep]['a'])
                a7 = float(data[rn2][lm7][timestep]['a'])
                a8 = float(data[rn2][lm8][timestep]['a'])
                a9 = float(data[rn2][lm9][timestep]['a'])
                a10 = float(data[rn2][lm10][timestep]['a'])

                mhr1 = (np.abs(aPL-a1))/((((massPL + data[rn2][lm][0]['mass'])/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + data[rn2][lm2][0]['mass'])/3) ** (1/3)) * ((aPL + a2)/2))
                mhr3 = (np.abs(aPL-a3))/((((massPL + data[rn2][lm3][0]['mass'])/3) ** (1/3)) * ((aPL + a3)/2))
                mhr4 = (np.abs(aPL-a4))/((((massPL + data[rn2][lm4][0]['mass'])/3) ** (1/3)) * ((aPL + a4)/2))
                mhr5 = (np.abs(aPL-a5))/((((massPL + data[rn2][lm5][0]['mass'])/3) ** (1/3)) * ((aPL + a5)/2))
                mhr6 = (np.abs(aPL-a6))/((((massPL + data[rn2][lm6][0]['mass'])/3) ** (1/3)) * ((aPL + a6)/2))
                mhr7 = (np.abs(aPL-a7))/((((massPL + data[rn2][lm7][0]['mass'])/3) ** (1/3)) * ((aPL + a7)/2))
                mhr8 = (np.abs(aPL-a8))/((((massPL + data[rn2][lm8][0]['mass'])/3) ** (1/3)) * ((aPL + a8)/2))
                mhr9 = (np.abs(aPL-a9))/((((massPL + data[rn2][lm9][0]['mass'])/3) ** (1/3)) * ((aPL + a9)/2))
                mhr10 = (np.abs(aPL-a10))/((((massPL + data[rn2][lm10][0]['mass'])/3) ** (1/3)) * ((aPL + a10)/2))


                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['mhr3'].append(float(mhr1))
                te[rn2][planet]['mhr4'].append(float(mhr2))
                te[rn2][planet]['mhr5'].append(float(mhr1))
                te[rn2][planet]['mhr6'].append(float(mhr2))
                te[rn2][planet]['mhr7'].append(float(mhr1))
                te[rn2][planet]['mhr8'].append(float(mhr2))
                te[rn2][planet]['mhr9'].append(float(mhr1))
                te[rn2][planet]['mhr10'].append(float(mhr2))

                te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
                te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
                te[rn2][planet]['log a3_ratio'].append(float(np.log10(aPL/a3)))
                te[rn2][planet]['log a4_ratio'].append(float(np.log10(aPL/a4)))
                te[rn2][planet]['log a5_ratio'].append(float(np.log10(aPL/a5)))
                te[rn2][planet]['log a6_ratio'].append(float(np.log10(aPL/a6)))
                te[rn2][planet]['log a7_ratio'].append(float(np.log10(aPL/a7)))
                te[rn2][planet]['log a8_ratio'].append(float(np.log10(aPL/a8)))
                te[rn2][planet]['log a9_ratio'].append(float(np.log10(aPL/a9)))
                te[rn2][planet]['log a10_ratio'].append(float(np.log10(aPL/a10)))
                
                
            except:
                pass

                        

var = ['e','i','mhr1','mhr2','mhr10','mhr3','mhr4','mhr5','mhr6','mhr7','mhr8','mhr9',
       'log a1_ratio','log a2_ratio','log a3_ratio','log a4_ratio','log a5_ratio','log a6_ratio',
      'log a7_ratio','log a8_ratio','log a9_ratio','log a10_ratio'] 

                
                
for rn2 in range(1,101):
    for planet in range (1,11):
        sys[rn2][planet]['cv a'] = np.std(te[rn2][planet]['a'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['a'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv e'] = np.std(te[rn2][planet]['e'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['e'][0:len(data[rn2][planet])])
        sys[rn2][planet]['cv i'] = np.std(te[rn2][planet]['i'][0:len(data[rn2][planet])])/np.mean(te[rn2][planet]['i'][0:len(data[rn2][planet])])
        
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

<ipython-input-24-0a65c4b39f03>:175: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a8_ratio'].append(float(np.log10(aPL/a8)))
<ipython-input-24-0a65c4b39f03>:168: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a1_ratio'].append(float(np.log10(aPL/a1)))
<ipython-input-24-0a65c4b39f03>:169: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a2_ratio'].append(float(np.log10(aPL/a2)))
<ipython-input-24-0a65c4b39f03>:170: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a3_ratio'].append(float(np.log10(aPL/a3)))
<ipython-input-24-0a65c4b39f03>:171: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a4_ratio'].append(float(np.log10(aPL/a4)))
<ipython-input-24-0a65c4b39f03>:172: RuntimeWarning: invalid value encountered in log10
  te[rn2][planet]['log a5_ratio'].append(float(np.log10(aPL/a5)))
<ipython-input-24-0a65c4b39f03>:173: RuntimeWarning: invalid value encounter

### dynamically closest - 3 planets

In [5]:
data = dict()

for rn in range(1,101):
    data[rn] = dict()
    for planetnum in range(1,11):
        data[rn][planetnum] = dict()
        pl = genfromtxt(r'/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn)+'/PL'+str(planetnum)+'.el',skip_header=2,max_rows=10)
        
        for time in range(len(pl)):
            data[rn][planetnum][time] = dict()
            try:
                data[rn][planetnum][time]['mass'] = pl[time][1]
                data[rn][planetnum][time]['a'] = pl[time][2]
                data[rn][planetnum][time]['e'] = pl[time][3]
                data[rn][planetnum][time]['i'] = pl[time][4]
                data[rn][planetnum][time]['jacobi distance'] = pl[time][8]
            except:
                data[rn][planetnum][time]['mass'] = pl[1]
                data[rn][planetnum][time]['a'] = pl[2]
                data[rn][planetnum][time]['e'] = pl[3]
                data[rn][planetnum][time]['i'] = pl[4]
                data[rn][planetnum][time]['jacobi distance'] = pl[8]
            

    
    
sys = dict()
te = dict()

# initial, final, mean, med, max, min, standard deviation for each over entire timestep  
var = ['e','i','pericenter','jacobi','mhr1','mhr2','mhr3','a1_ratio','a2_ratio','a3_ratio','mass1_ratio','mass2_ratio','mass3_ratio'] 

    
for rn2 in range(1,101):
    sys[rn2] = dict()
    te[rn2] = dict()
    
    for planet in range (1,11):
        sys[rn2][planet] = dict() 
        te[rn2][planet] = dict()
        
        te[rn2][planet]['e'] = []
        te[rn2][planet]['i'] = []
        te[rn2][planet]['pericenter'] = []
        te[rn2][planet]['jacobi'] = []
        te[rn2][planet]['mhr1'] = []
        te[rn2][planet]['mhr2'] = []
        te[rn2][planet]['mhr3'] = []
        te[rn2][planet]['a1_ratio'] = []
        te[rn2][planet]['a2_ratio'] = []
        te[rn2][planet]['a3_ratio'] = []
        te[rn2][planet]['mass1_ratio'] = []
        te[rn2][planet]['mass2_ratio'] = []
        te[rn2][planet]['mass3_ratio'] = []

    
        for timestep in range(len(data[rn2][planet])):
            try:
                te[rn2][planet]['e'].append(float(data[rn2][planet][timestep]['e']))
                te[rn2][planet]['i'].append(float(data[rn2][planet][timestep]['i']))
                te[rn2][planet]['jacobi'].append(float(data[rn2][planet][timestep]['jacobi distance']))

                massPL = float(data[rn2][planet][timestep]['mass'])
                aPL = float(data[rn2][planet][timestep]['a'])
                ePL = float(data[rn2][planet][timestep]['e'])
                
                
                fcmhr = []
                ps = []
                
                for p in range(1,11):
                    if p == planet:
                        pass
                    else:
                        ps.append(p)
                        fcmhr.append((np.abs(aPL-data[rn2][p][timestep]['a']))/((((massPL + data[rn2][p][timestep]['mass'])/3) ** (1/3)) * ((aPL + data[rn2][p][timestep]['a'])/2)))                        
                
                p1 = ps[fcmhr.index(np.max(fcmhr))]
                
                fcmhr.remove(np.max(fcmhr))
                ps.remove(p1)
                
                p2 = ps[fcmhr.index(np.max(fcmhr))]
                
                fcmhr.remove(np.max(fcmhr))
                ps.remove(p2)
                
                p3 = ps[fcmhr.index(np.max(fcmhr))]
                
                
                m1 = float(data[rn2][p1][timestep]['mass'])
                a1 = float(data[rn2][p1][timestep]['a'])

                m2 = float(data[rn2][p2][timestep]['mass'])
                a2 = float(data[rn2][p2][timestep]['a'])

                m3 = float(data[rn2][p3][timestep]['mass'])
                a3 = float(data[rn2][p3][timestep]['a'])
                
                
                mhr1 = (np.abs(aPL-a1))/((((massPL + m1)/3) ** (1/3)) * ((aPL + a1)/2))
                mhr2 = (np.abs(aPL-a2))/((((massPL + m2)/3) ** (1/3)) * ((aPL + a2)/2))
                mhr3 = (np.abs(aPL-a3))/((((massPL + m3)/3) ** (1/3)) * ((aPL + a3)/2))
                
                te[rn2][planet]['mhr1'].append(float(mhr1))
                te[rn2][planet]['mhr2'].append(float(mhr2))
                te[rn2][planet]['mhr3'].append(float(mhr3))
                te[rn2][planet]['a1_ratio'].append(float(aPL/a1))
                te[rn2][planet]['a2_ratio'].append(float(aPL/a2))
                te[rn2][planet]['a3_ratio'].append(float(aPL/a3))
                te[rn2][planet]['mass1_ratio'].append(float(massPL/m1))
                te[rn2][planet]['mass2_ratio'].append(float(massPL/m2))
                te[rn2][planet]['mass3_ratio'].append(float(massPL/m3))
                te[rn2][planet]['pericenter'].append(aPL*(1-ePL))
                
                
            except:
                pass

                        
for rn2 in range(1,101):
    for planet in range (1,11):
        for v in var:
            try:
                sys[rn2][planet]['initial ' + str(v)] = te[rn2][planet][str(v)][0]
                sys[rn2][planet]['final ' + str(v)] = te[rn2][planet][str(v)][-1]
                sys[rn2][planet]['mean ' + str(v)] = np.mean(te[rn2][planet][str(v)])  
                sys[rn2][planet]['min ' + str(v)] = np.amin(te[rn2][planet][str(v)])
                sys[rn2][planet]['max ' + str(v)] = np.amax(te[rn2][planet][str(v)])
                sys[rn2][planet]['sd ' + str(v)] = np.std(te[rn2][planet][str(v)])
            except:
                pass

                

# # instability time

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['end time'] = 10000000.0
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
    
                if "ejected" in line:
                    sys[rn2][plnt]['end time'] = float(cols[3])
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['end time'] = float(cols[7])
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['end time'] = float(cols[6])
                    
    
# final planet fate

for rn2 in range(1,101):
    for planet in range(1,11):
        sys[rn2][planet]['fate'] = 'remaining'
    
    with open('/Users/neha/Documents/lanl/classifier/s. mordasini data/run'+str(rn2)+'/info.out', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if "PL" in line:
                cols = line.split()
                plnt = int(cols[0].split('L')[-1])
                plnt2 = cols[4].split('L')[-1]
                
                if "ejected" in line:
                    sys[rn2][plnt]['fate'] = "ejected"
                elif "collided with the central body" in line:
                    sys[rn2][plnt]['fate'] = "central body collision"
                elif "was hit by" in line:
                    sys[rn2][int(plnt2)]['fate'] = "planet-planet collision"

### writing csv file

In [33]:
nested_keys = []
nested_keys.append('cv e')
nested_keys.append('cv a')
nested_keys.append('cv peri')
nested_keys.append('cv i')

# var = ['e','i','pericenter','jacobi','mhr1','a1_ratio','mass1_ratio'] 
# var = ['e','i','pericenter','jacobi','mhr1','mhr2','log a1_ratio','log a2_ratio','log mass1_ratio','log mass2_ratio'] 
var = ['e','i','pericenter','jacobi','mhr1','mhr2','mhr3','log a1_ratio','log a2_ratio','log a3_ratio','log mass1_ratio','log mass2_ratio','log mass3_ratio'] 

# var = ['e','i','mhr1','mhr2','mhr10','mhr3','mhr4','mhr5','mhr6','mhr7','mhr8','mhr9',
#        'log a1_ratio','log a2_ratio','log a3_ratio','log a4_ratio','log a5_ratio','log a6_ratio',
#       'log a7_ratio','log a8_ratio','log a9_ratio','log a10_ratio'] 


for v in var:
    nested_keys.append('initial ' + str(v))
    nested_keys.append('final ' + str(v))
    nested_keys.append('mean ' + str(v))
    nested_keys.append('min ' + str(v))
    nested_keys.append('max ' + str(v))
    nested_keys.append('sd ' + str(v))
    

# nested_keys.append('log mass1_ratio') 
# nested_keys.append('log mass2_ratio') 
# nested_keys.append('log mass3_ratio') 
# nested_keys.append('log mass4_ratio') 
# nested_keys.append('log mass5_ratio') 
# nested_keys.append('log mass6_ratio') 
# nested_keys.append('log mass7_ratio') 
# nested_keys.append('log mass8_ratio') 
# nested_keys.append('log mass9_ratio') 
# nested_keys.append('log mass10_ratio') 
    
    
with open('3 most massive adj mass.csv', 'w') as f:
    header = 'run, planet, ' + ', '.join(map(str,nested_keys)) + ', end time, fate' + '\n'
    f.write(header)
    
    for r in list(sys.keys()):
        for p in list(sys[r].keys()):
            line = ''
            line += str(r) + ', ' + str(p)
            for va in nested_keys:
                line += ', ' + str(sys[r][p][va])
            line += ', ' + str(sys[r][p]['end time']) + ', ' + str(sys[r][p]['fate'])
            line += '\n'
            f.write(line)